In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import adhtools.utils
import pandas as pd
import os
import numpy as np

## Load corpus

In [3]:
import glob

book_files = glob.glob('/media/sf_VBox_Shared/Arabic/Fiqh/2018-11-13-Fiqh-stemmed-chapters/*.xml')
fnames = [os.path.basename(fn) for fn in book_files]
print(len(book_files))

10066


In [4]:
path_to_stopwordlist = '/media/sf_VBox_Shared/Arabic/arabic_stop-words_7-8-2018.txt'
external_stopwords = [line.strip() for line in open(path_to_stopwordlist, 'r', encoding='utf-8')]

In [5]:
corpus = adhtools.utils.corpus_str(book_files, analyzer=False, field='proposed_root')

## Make document-term matrix

In [6]:
vectorizer = CountVectorizer( stop_words=external_stopwords, min_df=2, max_df=0.9)
X = vectorizer.fit_transform(corpus)

In [7]:
feature_names = vectorizer.get_feature_names()
len(feature_names)

29755

In [8]:
# some example words from the first document
np.array(feature_names)[X[0].indices][:50]

array(['بقل', 'ربط', 'جدي', 'رفق', 'شرط', 'خوص', 'منع', 'قطع', 'بلد',
       'غرر', 'وإنا', 'طوق', 'زخرف', 'قرن', 'ذلل', 'سخر', 'سيأ', 'فوت',
       'حول', 'وسي', 'مدي', 'همز', 'كوف', 'فتح', 'وكآبة', 'همل', 'بسأ',
       'وعث', 'عرف', 'حرف', 'سرد', 'فذلك', 'سبع', 'وسع', 'قعد', 'قيأ',
       'شخص', 'هيأ', 'مضطجع', 'قذر', 'لحم', 'بات', 'أجر', 'عطي', 'فنادق',
       'فهم', 'غرب', 'رخص', 'رغث', 'قمل'], dtype='<U19')

In [9]:
most_imp_words = pd.Series(np.array(feature_names)[X.argmax(axis=1)].flatten())
most_imp_words.value_counts().head(20)

صلي    563
سلم    477
بيع    412
شهد    242
طلق    202
حرم    192
قتل    164
ملك    148
كتب    145
عتق    143
زكي    130
صوم    121
زوج    115
وصي    112
قرر    111
رهن    107
أبن    107
ثلث    105
دعا     99
ولد     98
dtype: int64

## Train model

In [13]:
nr_topics = 10

In [14]:
import lda
model = lda.LDA(n_topics=nr_topics, n_iter=1000)
model.fit(X)

INFO:lda:n_documents: 10066
INFO:lda:vocab_size: 29755
INFO:lda:n_words: 25060379
INFO:lda:n_topics: 10
INFO:lda:n_iter: 1000
INFO:lda:<0> log likelihood: -225944102
INFO:lda:<10> log likelihood: -211962484
INFO:lda:<20> log likelihood: -188614270
INFO:lda:<30> log likelihood: -185684903
INFO:lda:<40> log likelihood: -184884436
INFO:lda:<50> log likelihood: -184470324
INFO:lda:<60> log likelihood: -184194843
INFO:lda:<70> log likelihood: -183947053
INFO:lda:<80> log likelihood: -183666613
INFO:lda:<90> log likelihood: -183479900
INFO:lda:<100> log likelihood: -183356511
INFO:lda:<110> log likelihood: -183267991
INFO:lda:<120> log likelihood: -183195951
INFO:lda:<130> log likelihood: -183146336
INFO:lda:<140> log likelihood: -183096768
INFO:lda:<150> log likelihood: -183064542
INFO:lda:<160> log likelihood: -183023691
INFO:lda:<170> log likelihood: -182983450
INFO:lda:<180> log likelihood: -182963574
INFO:lda:<190> log likelihood: -182916416
INFO:lda:<200> log likelihood: -182886115
INF

In [9]:
# lda = LatentDirichletAllocation(n_components=nr_topics, random_state=0, max_iter=50)
# document_topics = lda.fit_transform(X)

In [15]:
document_topics = model.doc_topic_

## Save files

In [16]:
fp_out = '/media/sf_VBox_Shared/Arabic/Analyses/topicmodelling/'

In [17]:
df_document_topics = pd.DataFrame(document_topics, index=fnames)
df_document_topics.to_csv(os.path.join(fp_out, 'fiqh_roots_document_topics_{}.csv'.format(nr_topics)))

In [18]:
#lda.components_.shape
model.components_.shape

(10, 29755)

In [19]:
topic_words = pd.DataFrame(np.argsort(model.components_, axis=1)[:,-10:][::-1])

In [20]:
topic_words = topic_words.applymap(lambda l: feature_names[l])

In [21]:
topic_words.to_csv(os.path.join(fp_out, 'fiqh_roots_topics_{}.csv'.format(nr_topics)))

In [22]:
topic_words

,0,1,2,3,4,5,6,7,8,9
0,وبأ,علم,صحي,الخ,شرح,خلف,ظهر,ذكر,غور,اه
1,عبر,علم,ذكر,خبر,خلف,عرف,ظهر,سلم,غور,عدم
2,روي,عمر,نبأ,حمد,فى,رسل,أبن,صلي,حدث,سلم
3,ورث,ربع,كتب,ولد,موت,عتق,قسم,وصي,ثلث,أبن
4,غور,وجب,حلل,كفر,جزأ,حوج,يوم,أكل,صوم,حرم
5,ثبت,قوم,شهد,رهن,بيع,ملك,قرر,دعا,قضي,حقق
6,عمل,ثمن,شرط,عقد,أخذ,أجر,ملك,جوز,شري,بيع
7,غور,خلف,طهر,قوم,سلم,ركع,أمم,غسل,سجد,صلي
8,ثني,ثلث,عدا,فلي,ولد,مرأ,وطأ,نكح,زوج,طلق
9,فلي,حدد,قوم,جني,سلم,حكم,ودي,قطع,شهد,قتل
